# Results

In this notebook we explore the results of the experiments, which are stored in wandb

In [26]:
# Get wandb api object
import wandb

api = wandb.Api()

# Get all runs
project = api.project("pysentimiento")


# Get all runs for sentiment and lang es
runs = [r for r in api.runs("pysentimiento", {"config.task": "targeted_sa"}) if not r.sweep]



In [27]:
len(runs)

209

## Spanish results

In [28]:
data = []

renames = {
    "PlanTL-GOB-ES/roberta-base-bne": "roberta",
    "bertin-project/bertin-roberta-base-spanish": "bertin",
    "dccuchile/bert-base-spanish-wwm-cased": "beto",
    "mrm8488/electricidad-base-discriminator": "electricidad",
    "pysentimiento/robertuito-base-uncased": "robertuito",
}

for run in runs:
    # Get model name
    model_name = run.config["model"]
    # Get task 
    task = run.config["task"]
    # Get Macro f1 performance
    try:
        pos_f1 = run.summary["test_pos_f1"]
        neg_f1 = run.summary["test_neg_f1"]
        neu_f1 = run.summary["test_neu_f1"]
        macro_f1 = run.summary["test_macro_f1"]
    except KeyError as e:
        print(f"Skipping {model_name} {task} because it does not have all metrics")
        print(e)
        continue
    data.append({
        "model": model_name,
        "macro_f1": macro_f1,
        "pos_f1": pos_f1,
        "neg_f1": neg_f1,
        "neu_f1": neu_f1,
    })

Skipping PlanTL-GOB-ES/roberta-base-bne_untargeted targeted_sa because it does not have all metrics
'test_pos_f1'
Skipping dccuchile/bert-base-spanish-wwm-cased_untargeted targeted_sa because it does not have all metrics
'test_pos_f1'
Skipping pysentimiento/robertuito-base-uncased_untargeted targeted_sa because it does not have all metrics
'test_pos_f1'
Skipping dccuchile/bert-base-spanish-wwm-cased targeted_sa because it does not have all metrics
'test_pos_f1'
Skipping mrm8488/electricidad-base-discriminator targeted_sa because it does not have all metrics
'test_pos_f1'
Skipping dccuchile/bert-base-spanish-wwm-cased targeted_sa because it does not have all metrics
'test_pos_f1'
Skipping pysentimiento/robertuito-base-uncased targeted_sa because it does not have all metrics
'test_pos_f1'
Skipping pysentimiento/robertuito-base-uncased targeted_sa because it does not have all metrics
'test_pos_f1'
Skipping PlanTL-GOB-ES/roberta-base-bne targeted_sa because it does not have all metrics
'te

In [30]:
import pandas as pd

df = pd.DataFrame(data)

# Group by model and task

def rename_model(model_name):
    if model_name.endswith("_untargeted"):
        model, _ = model_name.split("_untargeted")
        return f"{renames[model]} (untargeted)"
    else:
        return renames[model_name]

df["model"] = df["model"].map(rename_model)
grouped = df.groupby(["model"])

# Get mean macro f1 plus standard deviation in a single line with +- inbetween

df_results = (grouped.mean() * 100).round(1).astype(str) + " +- " + (grouped.std() * 100).round(1).astype(str)

#df_results = df_results.unstack("task")
#df_results.columns = df_results.columns.droplevel(0)

df_results[["pos_f1", "neg_f1", "neu_f1", "macro_f1"]]

,pos_f1,neg_f1,neu_f1,macro_f1
model,,,,
bertin,67.3 +- 5.6,63.6 +- 5.5,58.0 +- 5.5,63.0 +- 4.7
bertin (untargeted),68.6 +- 3.6,65.3 +- 4.5,63.2 +- 4.6,65.7 +- 3.3
beto,73.8 +- 2.1,72.9 +- 3.3,65.9 +- 4.5,70.9 +- 2.3
beto (untargeted),70.7 +- 3.7,66.9 +- 4.4,64.4 +- 3.8,67.3 +- 3.1
electricidad,67.3 +- 1.3,63.1 +- 6.0,55.2 +- 4.9,61.9 +- 2.7
electricidad (untargeted),68.0 +- 2.8,61.4 +- 5.5,59.3 +- 3.5,62.9 +- 3.0
roberta,75.3 +- 3.9,74.2 +- 2.9,68.5 +- 4.7,72.7 +- 2.6
roberta (untargeted),72.3 +- 3.5,68.6 +- 4.5,66.8 +- 4.0,69.3 +- 2.9
robertuito,75.0 +- 2.6,73.8 +- 2.3,67.8 +- 3.2,72.2 +- 1.4


In [31]:
print(df_results.to_markdown())

| model                     | macro_f1    | pos_f1      | neg_f1      | neu_f1      |
|:--------------------------|:------------|:------------|:------------|:------------|
| bertin                    | 63.0 +- 4.7 | 67.3 +- 5.6 | 63.6 +- 5.5 | 58.0 +- 5.5 |
| bertin (untargeted)       | 65.7 +- 3.3 | 68.6 +- 3.6 | 65.3 +- 4.5 | 63.2 +- 4.6 |
| beto                      | 70.9 +- 2.3 | 73.8 +- 2.1 | 72.9 +- 3.3 | 65.9 +- 4.5 |
| beto (untargeted)         | 67.3 +- 3.1 | 70.7 +- 3.7 | 66.9 +- 4.4 | 64.4 +- 3.8 |
| electricidad              | 61.9 +- 2.7 | 67.3 +- 1.3 | 63.1 +- 6.0 | 55.2 +- 4.9 |
| electricidad (untargeted) | 62.9 +- 3.0 | 68.0 +- 2.8 | 61.4 +- 5.5 | 59.3 +- 3.5 |
| roberta                   | 72.7 +- 2.6 | 75.3 +- 3.9 | 74.2 +- 2.9 | 68.5 +- 4.7 |
| roberta (untargeted)      | 69.3 +- 2.9 | 72.3 +- 3.5 | 68.6 +- 4.5 | 66.8 +- 4.0 |
| robertuito                | 72.2 +- 1.4 | 75.0 +- 2.6 | 73.8 +- 2.3 | 67.8 +- 3.2 |
| robertuito (untargeted)   | 69.7 +- 2.5 | 72.6 +- 2.

In [32]:
print(df_results.to_latex())

\begin{tabular}{lllll}
\toprule
{} &     macro\_f1 &       pos\_f1 &       neg\_f1 &       neu\_f1 \\
model                     &              &              &              &              \\
\midrule
bertin                    &  63.0 +- 4.7 &  67.3 +- 5.6 &  63.6 +- 5.5 &  58.0 +- 5.5 \\
bertin (untargeted)       &  65.7 +- 3.3 &  68.6 +- 3.6 &  65.3 +- 4.5 &  63.2 +- 4.6 \\
beto                      &  70.9 +- 2.3 &  73.8 +- 2.1 &  72.9 +- 3.3 &  65.9 +- 4.5 \\
beto (untargeted)         &  67.3 +- 3.1 &  70.7 +- 3.7 &  66.9 +- 4.4 &  64.4 +- 3.8 \\
electricidad              &  61.9 +- 2.7 &  67.3 +- 1.3 &  63.1 +- 6.0 &  55.2 +- 4.9 \\
electricidad (untargeted) &  62.9 +- 3.0 &  68.0 +- 2.8 &  61.4 +- 5.5 &  59.3 +- 3.5 \\
roberta                   &  72.7 +- 2.6 &  75.3 +- 3.9 &  74.2 +- 2.9 &  68.5 +- 4.7 \\
roberta (untargeted)      &  69.3 +- 2.9 &  72.3 +- 3.5 &  68.6 +- 4.5 &  66.8 +- 4.0 \\
robertuito                &  72.2 +- 1.4 &  75.0 +- 2.6 &  73.8 +- 2.3 &  67.8 +- 3.2 \\

/tmp/ipykernel_1905181/2638068283.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df_results.to_latex())
